<a href="https://colab.research.google.com/github/thedanindanger/yaads-examples/blob/dev/zipCodeImpute/ZIPCodeImpute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
BigQuery has several datasets from the census based on SCTA and ZIP Codes.

The problem is that ZCTA from census and ZIP are not a perfect match. ZCTA is only high population 

In another example, I created a notebook pulling census population ZIP codes, then finding the state average ZIP population to impute missing values: https://github.com/thedanindanger/yaads-examples/tree/main/ColabIntro 

However, there is likely a much better solution. First I will try a hierarchical outer join*, then I will try a nearest neighbor imputation given the cartesian distance between ZIP code centroid locations.

*Note: I made this term up as far as I know, the explanation is at the end.

#Connect to BigQuery


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
##@title GCP Project Name
project_id = "" #@param {type:"string"}
%load_ext google.colab.data_table

In [ ]:
%%bigquery --project $project_id zip_pop_df
SELECT 
  --Forces leading zeros, e.g. ZIP 34 would first concat to 0000034,
  --  Then the right five are 00034
  right(concat('00000' ,cast(zipcode as string)),5) as zip_5, 
  sum(population) as population 
 FROM `bigquery-public-data.census_bureau_usa.population_by_zip_2010` 
 GROUP BY 1

#Explore missing values
Now we have the same set of ZIPs as our previous example.

Let's see how many ZIPs we are missing.

In [ ]:
missing_zips = zip_pop_df[zip_pop_df['population'].isnull()]

In [ ]:
missing_zips

,zip_5,population


Do you see the problem? We don't know which ZIPs are missing because we have nothing to compare them against.

In the previous example, there were a theorectical list of customers. 

I do have a list of ZIP codes which I need to assign population; however, I will be aggregating the population by DMA, which is a proprietary aggregation measure and one I am not at liberty to disclose. Therefore, I will only include the list of ZIPs I need.



In [ ]:
import pandas as pd

zip_url = 'https://raw.githubusercontent.com/thedanindanger/yaads-examples/main/zipCodeImpute/target_zips.csv'

target_zips = pd.read_csv(zip_url)

In [ ]:
target_zips.head()

,Zip
0,79699
1,79698
2,79697
3,79606
4,79605


In [ ]:
#zero padding or zero filling. Adds up to 5 zero as first digits. Example: 00313  instead of 313
zip_pop_df['zip_5'] = zip_pop_df['zip_5'].str.zfill(5)

#target_zips read as integer, so first convert to string
target_zips['Zip'] = target_zips['Zip'].astype(str).str.zfill(5)

In [ ]:
target_zips['Zip']
test_zips = pd.merge(target_zips, zip_pop_df, left_on='Zip', right_on='zip_5', how='left')
missing_zips = test_zips[test_zips['zip_5'].isnull()]

In [ ]:
missing_zips

,Zip,zip_5,population
1,79698,NaN,NaN
2,79697,NaN,NaN
89,31704,NaN,NaN
111,31760,NaN,NaN
417,87131,NaN,NaN
...,...,...,...
29479,67476,NaN,NaN
29784,17877,NaN,NaN
29929,00073,NaN,NaN
29940,00074,NaN,NaN


In [ ]:
missing_count = missing_zips['Zip'].count()
total_count = target_zips['Zip'].count()

print(f'Missing {missing_count} of {total_count} ZIP codes; or {round((missing_count / total_count) *100,2)}% of total target ZIPs')


Missing 447 of 30056 ZIP codes; or 1.49% of total target ZIPs


#Hierarchical backfill of missing ZIPs
Now we know there are still ZIPs missing. That means we can process our data a little more rigorously.

At this point, I will load the data into Bigquery for further evaluation, mostly because SQL is generally considered easier to read and more widely understood than Python.

I created a table in BigQuery through the cloud console. It's very easy to do. I plan to make a video on it one day. If this still isn't updated feel free to make a comment on the repo to remind me.

You can use whatever name you like, but the region needs to be 'us' since the google public data is in that region as well https://cloud.google.com/bigquery/docs/locations 

In [ ]:
#@title BigQuery Dataset and Table
bq_dataset = "sample_data" #@param {type:"string"}
bq_table = "target_zips" #@param {type:"string"}
bq_region = "us" #@param {type:"string"}



In [ ]:
import pandas_gbq
pandas_gbq.to_gbq(
    target_zips, f'{bq_dataset}.{bq_table}', project_id=project_id, if_exists='replace',location=bq_region
)

1it [00:02,  2.96s/it]


In [ ]:
%%bigquery --project $project_id 
select * from `yaads-articles.sample_data.target_zips`
limit 5 

,Zip
0,79699
1,79698
2,79697
3,79606
4,79605


With that loaded, we can get down to business.

Census has tons of zip data sets, we can try several of the most recent.

In [ ]:
%%bigquery population_zips_multi_census --project $project_id 
select 
  distinct
  t.zip,
  ifnull(z10.population, 
    ifnull(z18.total_pop,
      ifnull(z17.total_pop,
        ifnull(z16.total_pop,
          ifnull(z15.total_pop,
            ifnull(z14.total_pop, NULL)
            )
          )
        ) 
      )
    ) as population
from
  `yaads-articles.sample_data.target_zips` t
  left join
    (
  --NOTE: Census had a lot of duplication
    select
      right(concat('00000',zipcode),5) as zipcode,
      max(population) as population
    from
      `bigquery-public-data.census_bureau_usa.population_by_zip_2010`
    group by 1
  ) z10
  on t.zip = right(concat('00000',z10.zipcode),5) --census 2010 zipcode is not zero padded
  left join
  `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr` z18
  on t.zip = z18.geo_id --'geo_id' is ZIP code in zip acs tables
  left join
  `bigquery-public-data.census_bureau_acs.zip_codes_2017_5yr` z17
  on t.zip = z17.geo_id
  left join
  `bigquery-public-data.census_bureau_acs.zip_codes_2016_5yr` z16
  on t.zip = z16.geo_id
  left join
  `bigquery-public-data.census_bureau_acs.zip_codes_2015_5yr` z15
  on t.zip = z15.geo_id
  left join
  `bigquery-public-data.census_bureau_acs.zip_codes_2014_5yr` z14
  on t.zip = z14.geo_id

In [ ]:
missing_zips_multi_census = population_zips_multi_census[population_zips_multi_census['population'].isnull()]

In [ ]:
missing_zips_multi_census.count()

zip           447
population      0
dtype: int64

#Imputing by distance 
Well that was completely useless...

One last thing to try. There is a 'shape' file in BigQuery containing all the ZIP code boundaries in the US. Perhaps we can find a match there, then impute based on close neighboring ZIP populations.

In [ ]:
%%bigquery zips_geo_join --project $project_id
select
zip,
internal_point_lat as lat,
internal_point_lon as lon
from 
  `yaads-articles.sample_data.target_zips` t
left join 
  `bigquery-public-data.geo_us_boundaries.zip_codes` g
on t.zip = g.zip_code

In [ ]:
missing_zips_geo_join = zips_geo_join[zips_geo_join['lat'].isnull()]

In [ ]:
missing_zips_geo_join.count()

zip    475
lat      0
lon      0
dtype: int64

Google is still using census data for this, so now for the big guns:
http://download.geonames.org/export/zip/

A repo of all postal codes in the world.

Credit to www.geonames.org for maintaining the repo

While it is relatively simple to upload a csv into BigQuery, these files do not include headers, so you will need to make your own schema.  I made a quick script for you to create the empty table yourself.

Be sure to replace the project name and dataset with your own

In [ ]:
%%bigquery --project $project_id
create table if not exists `yaads-articles.sample_data.zip_us_geo`
(country_code	STRING(2),
zip	STRING(5),
place_name	STRING(180),
admin_name1	STRING(100),
admin_code1	STRING(20),
admin_name2	STRING(100),
admin_code2	STRING(20),
admin_name3	STRING(100),
admin_code3	STRING(20),
lat	NUMERIC,
long	NUMERIC,
accuracy	STRING
)

In [ ]:
%%bigquery --project $project_id
select * from  `yaads-articles.sample_data.zip_us_geo`
order by zip asc
limit 100

,country_code,zip,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,lat,long,accuracy
0,US,00501,Holtsville,New York,NY,Suffolk,103,None,None,40.8154,-73.0451,4
1,US,00544,Holtsville,New York,NY,Suffolk,103,None,None,40.8154,-73.0451,4
2,US,01001,Agawam,Massachusetts,MA,Hampden,013,None,None,42.0702,-72.6227,4
3,US,01002,Amherst,Massachusetts,MA,Hampshire,015,None,None,42.3671,-72.4646,4
4,US,01003,Amherst,Massachusetts,MA,Hampshire,015,None,None,42.3919,-72.5248,4
...,...,...,...,...,...,...,...,...,...,...,...,...
95,US,01202,Pittsfield,Massachusetts,MA,Berkshire,003,None,None,42.3929,-73.2285,4
96,US,01203,Pittsfield,Massachusetts,MA,Berkshire,003,None,None,42.3929,-73.2285,4
97,US,01220,Adams,Massachusetts,MA,Berkshire,003,None,None,42.6223,-73.1172,4
98,US,01222,Ashley Falls,Massachusetts,MA,Berkshire,003,None,None,42.0596,-73.3202,4


Notice it does consider the lat-longs to be strings, despite me telling it otherwise...

Let's test if the match actually worked and if the conversion to numeric worked as well.

In [ ]:
%%bigquery --project $project_id
select t.zip, 
  z10.population,
  cast(g.lat as numeric) as lat,
  cast(g.long as numeric) as long
from  
  `yaads-articles.sample_data.target_zips` t
left join
  `yaads-articles.sample_data.zip_us_geo` g
on right(concat('00000',t.zip),5)= right(concat('00000',g.zip),5)
left join (
  --NOTE: Census had a lot of duplication
    select
      right(concat('00000',zipcode),5) as zipcode,
      max(population) as population
    from
      `bigquery-public-data.census_bureau_usa.population_by_zip_2010`
    group by 1
 ) z10
  on right(concat('00000',t.zip),5) = right(concat('00000',z10.zipcode),5) 
where lat is null and z10.population is null

,zip,population,lat,long
0,00064,None,None,None
1,00198,None,None,None
2,00011,None,None,None
3,00006,None,None,None
4,00004,None,None,None
...,...,...,...,...
98,00087,None,None,None
99,00053,None,None,None
100,00067,None,None,None
101,00073,None,None,None


Well at least we are down to around a 100 unknown. And these places seem pretty remote, so we should be okay. In a real scenario we would check each one, then manually look up the population if there was one available. 100 points to check is not a whole lot.

In [ ]:
%%bigquery zips_df --project $project_id
select t.zip, 
  z10.population,
  cast(g.lat as numeric) as lat,
  cast(g.long as numeric) as long
from  
  `yaads-articles.sample_data.target_zips` t
left join
  `yaads-articles.sample_data.zip_us_geo` g
on right(concat('00000',t.zip),5)= right(concat('00000',g.zip),5)
left join (
  --NOTE: Census had a lot of duplication
    select
      right(concat('00000',zipcode),5) as zipcode,
      max(population) as population
    from
      `bigquery-public-data.census_bureau_usa.population_by_zip_2010`
    group by 1
 ) z10
  on right(concat('00000',t.zip),5) = right(concat('00000',z10.zipcode),5)
where g.lat is not null and g.long is not null  --need these for imputation

#Imputation
Now we have a data set with 4 columns:
* zip: our key value
* Lat and Long: values always present and create a cartesian matrix of x and y coordinates
* Population: our target value, some of which are missing

We want to approximate population based on the lat long values of known ZIP code population's closest to our unknown.

Luckily, this is literally the entire purpose of KNN (K-Nearest-Neighbors) imputation.

Since we are not missing any other values beside population, and lat-long share the same scale, KNN fits this use-case very well.

And SKLearn has it built right in.

https://scikit-learn.org/stable/modules/impute.html#nearest-neighbors-imputation

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer

#only include the columns for imputation
cols = ['population', 'lat','long']
imp_df = zips_df[cols]

#'Distance' will weight population values of closer ZIPs higher than ones further away
imputer = KNNImputer(n_neighbors=5, weights='distance',add_indicator=True)

post_imp = imputer.fit_transform(imp_df)




In [ ]:
post_imp_df = pd.DataFrame(post_imp)
post_imp_df.columns = ['population', 'lat','long','imp_flag']

In [ ]:
post_imp_df["zip"] = zips_df["zip"]

In [ ]:
post_imp_df.head()

,population,lat,long,imp_flag,zip
0,321.0,32.4665,-99.7117,0.0,79699
1,22885.0,32.3920,-99.7746,0.0,79606
2,30098.0,32.4320,-99.7724,0.0,79605
3,24288.0,32.4679,-99.7619,0.0,79603
4,21519.0,32.4178,-99.7214,0.0,79602


One last sanity check to make sure everything matches.

If our merge imputation was successful and are merge worked as planned, we should be able to match our zips and have a data set showing no population on the original set, and imputed data in the new one - while lat and long still match of course.

In [ ]:
merge_check = pd.merge(post_imp_df, zips_df, left_on="zip", right_on="zip")

In [ ]:
merge_check.head()

,population_x,lat_x,long_x,imp_flag,zip,population_y,lat_y,long_y
0,321.0,32.4665,-99.7117,0.0,79699,321.0,32.4665,-99.7117
1,22885.0,32.3920,-99.7746,0.0,79606,22885.0,32.392,-99.7746
2,30098.0,32.4320,-99.7724,0.0,79605,30098.0,32.432,-99.7724
3,24288.0,32.4679,-99.7619,0.0,79603,24288.0,32.4679,-99.7619
4,21519.0,32.4178,-99.7214,0.0,79602,21519.0,32.4178,-99.7214


In [ ]:
merge_check[merge_check["population_x"] != merge_check["population_y"]]

,population_x,lat_x,long_x,imp_flag,zip,population_y,lat_y,long_y
29610,19036.715927,32.4751,-99.7348,1.0,79698,NaN,32.4751,-99.7348
29611,19413.250963,32.4487,-99.7331,1.0,79697,NaN,32.4487,-99.7331
29612,29947.069090,31.5500,-84.0612,1.0,31704,NaN,31.55,-84.0612
29613,6359.234706,31.7063,-83.4086,1.0,31760,NaN,31.7063,-83.4086
29614,45004.937945,35.0443,-106.6729,1.0,87131,NaN,35.0443,-106.6729
...,...,...,...,...,...,...,...,...
29949,3901.528321,37.2377,-96.8389,1.0,67102,NaN,37.2377,-96.8389
29950,21725.850969,37.6066,-97.2979,1.0,67221,NaN,37.6066,-97.2979
29951,1077.589153,38.5508,-97.4303,1.0,67476,NaN,38.5508,-97.4303
29952,6984.846405,40.8790,-76.6673,1.0,17877,NaN,40.879,-76.6673
